In [3]:
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image

# ✅ 경로 설정
model_path = r"C:\Users\user\OneDrive\Desktop\Resnet182-real\model\model3newcopy2.pth"
json_dir = r"C:\Users\user\OneDrive\Desktop\Resnet182-real\jsonnew"

train_json_path = os.path.join(json_dir, "trainqwercopy2.json")
val_json_path = os.path.join(json_dir, "valqwercopy2.json")
test_json_path = os.path.join(json_dir, "testqwercopy2.json")

# ✅ 데이터 전처리 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ JSON 데이터 로드
def load_json_data(json_path):
    with open(json_path, "r") as f:
        return json.load(f)

train_data = load_json_data(train_json_path)
val_data = load_json_data(val_json_path)
test_data = load_json_data(test_json_path)

# ✅ 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

# ✅ 데이터 로더 생성
batch_size = 32
train_dataset = CustomDataset(train_data, transform)
val_dataset = CustomDataset(val_data, transform)
test_dataset = CustomDataset(test_data, transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# ✅ 모델 로드 및 가중치 적용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = efficientnet_b0(weights=None)  # 기존 모델 구조
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(set([item[1] for item in train_data])))  # 클래스 개수 조정
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

# ✅ 정확도 평가 함수
def evaluate(model, dataloader):
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return correct, total, accuracy

# ✅ 각 데이터셋별 평가 실행
train_correct, train_total, train_acc = evaluate(model, train_loader)
val_correct, val_total, val_acc = evaluate(model, val_loader)
test_correct, test_total, test_acc = evaluate(model, test_loader)

# ✅ 결과 출력
print(f"📌 Train 정확도: {train_correct}/{train_total} ({train_acc * 100:.2f}%)")
print(f"📌 Val 정확도: {val_correct}/{val_total} ({val_acc * 100:.2f}%)")
print(f"📌 Test 정확도: {test_correct}/{test_total} ({test_acc * 100:.2f}%)")


📌 Train 정확도: 4280/4281 (99.98%)
📌 Val 정확도: 1170/1229 (95.20%)
📌 Test 정확도: 574/606 (94.72%)


In [5]:
import os
import json
import shutil
import torch
import torch.nn as nn  # ✅ 오류 해결: torch.nn 추가
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image

# ✅ 경로 설정
json_dir = r"C:\Users\user\OneDrive\Desktop\Resnet182-real\jsonnew"
recall_root = r"C:\Users\user\OneDrive\Desktop\Resnet182-real\recall"
misclassified_root = os.path.join(recall_root, "misclassified_data")
original_dataset_root = r"C:\Users\user\OneDrive\Desktop\Resnet182-real\data\category_data3"

# ✅ JSON 데이터 로드
def load_json_data(json_path):
    with open(json_path, "r") as f:
        return json.load(f)

train_data = load_json_data(os.path.join(json_dir, "trainqwer.json"))
val_data = load_json_data(os.path.join(json_dir, "valqwer.json"))
test_data = load_json_data(os.path.join(json_dir, "testqwer.json"))

print(f"✅ 데이터 로드 완료: Train({len(train_data)}), Val({len(val_data)}), Test({len(test_data)})")

# ✅ 데이터 변환 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ 데이터셋 클래스 정의 (파일 경로 포함)
class CustomDatasetWithPaths(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label, img_path

# ✅ 데이터로더 생성
batch_size = 32
train_loader_paths = DataLoader(CustomDatasetWithPaths(train_data, transform), batch_size=batch_size, shuffle=False)
val_loader_paths = DataLoader(CustomDatasetWithPaths(val_data, transform), batch_size=batch_size, shuffle=False)
test_loader_paths = DataLoader(CustomDatasetWithPaths(test_data, transform), batch_size=batch_size, shuffle=False)

# ✅ 모델 로드 및 평가 함수
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = r"C:\Users\user\OneDrive\Desktop\Resnet182-real\model\model3new.pth"
model = efficientnet_b0(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(set([item[1] for item in train_data])))  # ✅ 오류 해결
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()

# ✅ 틀린 이미지 저장 함수
def save_misclassified_images(model, dataloader, dataset_name):
    model.eval()
    misclassified_count = 0

    with torch.no_grad():
        for images, labels, paths in tqdm(dataloader, desc=f"🔍 {dataset_name} 데이터셋 오류 이미지 저장"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for img_path, pred, label in zip(paths, predicted.cpu().numpy(), labels.cpu().numpy()):
                if pred != label:
                    # ✅ 원본 디렉토리 구조 유지하여 저장
                    relative_path = os.path.relpath(img_path, original_dataset_root)
                    save_path = os.path.join(misclassified_root, relative_path)

                    # ✅ 폴더 생성 후 이미지 복사
                    os.makedirs(os.path.dirname(save_path), exist_ok=True)
                    shutil.copy(img_path, save_path)
                    misclassified_count += 1

    print(f"✅ {dataset_name}: 총 {misclassified_count}개의 틀린 이미지 저장 완료!")

# ✅ 틀린 이미지 저장 실행
save_misclassified_images(model, train_loader_paths, "Train")
save_misclassified_images(model, val_loader_paths, "Validation")
save_misclassified_images(model, test_loader_paths, "Test")


✅ 데이터 로드 완료: Train(4767), Val(1368), Test(675)


🔍 Train 데이터셋 오류 이미지 저장: 100%|██████████| 149/149 [01:00<00:00,  2.47it/s]


✅ Train: 총 0개의 틀린 이미지 저장 완료!


🔍 Validation 데이터셋 오류 이미지 저장: 100%|██████████| 43/43 [00:17<00:00,  2.48it/s]


✅ Validation: 총 86개의 틀린 이미지 저장 완료!


🔍 Test 데이터셋 오류 이미지 저장: 100%|██████████| 22/22 [00:08<00:00,  2.64it/s]

✅ Test: 총 38개의 틀린 이미지 저장 완료!
